In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('crftag1000.tsv', encoding='latin1')

In [4]:
data = data.fillna(method="ffill")

In [19]:
data.tail(5)

,Sentence #,Word,POS,Tag
92764,Sentence: 1000,good,JJ,o
92765,Sentence: 1000,practice,NN,o
92766,Sentence: 1000,is,VBZ,o
92767,Sentence: 1000,welcome,JJ,o
92768,Sentence: 1000,.,.,o


In [6]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

10044

In [7]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

22

In [10]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(data)
sent = getter.get_next()

In [11]:
print(sent)

[('I', 'PRP', 'o'), ('want', 'VBP', 'o'), ('to', 'TO', 'o'), ('be', 'VB', 'o'), ('able', 'JJ', 'o'), ('to', 'TO', 'o'), ('display', 'VB', 'o'), ('a', 'DT', 'o'), ('normal', 'JJ', 'o'), ('YouTube', 'NNP', 'o'), ('video', 'NN', 'o'), ('with', 'IN', 'o'), ('overlaid', 'JJ', 'o'), ('annotations', 'NNS', 'o'), (',', ',', 'o'), ('consisting', 'VBG', 'o'), ('of', 'IN', 'o'), ('coloured', 'JJ', 'o'), ('rectangles', 'NNS', 'o'), ('for', 'IN', 'o'), ('each', 'DT', 'o'), ('frame', 'NN', 'o'), ('.', '.', 'o'), ('The', 'DT', 'o'), ('only', 'JJ', 'o'), ('requirement', 'NN', 'o'), ('is', 'VBZ', 'o'), ('that', 'IN', 'o'), ('this', 'DT', 'o'), ('should', 'MD', 'o'), ('be', 'VB', 'o'), ('done', 'VBN', 'o'), ('programmatically', 'RB', 'o'), ('.', '.', 'o'), ('YouTube', 'NNP', 'o'), ('has', 'VBZ', 'o'), ('annotations', 'NNS', 'o'), ('now', 'RB', 'o'), (',', ',', 'o'), ('but', 'CC', 'o'), ('require', 'VBP', 'o'), ('you', 'PRP', 'o'), ('to', 'TO', 'o'), ('use', 'VB', 'o'), ('their', 'PRP$', 'o'), ('front', 

In [12]:
sentences = getter.sentences

In [14]:
max_len = 150
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [15]:
word2idx["python"]

8268

In [20]:
tag2idx["SoftwareOperation"]

12

In [21]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

C:\Users\ALK4KOR\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [25]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)

In [28]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["o"])

In [29]:
from keras.utils import to_categorical

In [30]:
y = [to_categorical(i, num_classes=n_tags) for i in y]


In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)


In [33]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [34]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [35]:
model = Model(input, out)

In [36]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [ ]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=50,
                    validation_split=0.1, verbose=1)